In [1]:
# Import packages and initialize Earth Engine

import ee
import geemap
import pandas as pd
import numpy as np
import os
import seaborn as sns

geemap.ee_initialize()


Successfully saved authorization token.


In [2]:
# AUTO INPUTS
# print(os.getcwd())

#TYPE DATE HERE 
date = '10/06/2019'
input = pd.read_csv('../GEM/GEM_validationdata_input.csv')
# print(input.head(10))

# Extract values from input for the row with date = date
row = input[input['Datetime'].str.contains(date)]
# print(row)
id = row['Image_ID'].values[0]
AirT = row['Avg_AT'].values[0]
LRI = row['LRI_CB'].values[0]
avg_SRI = row['Avg_SRI'].values[0]
p = row['p'].values[0]

print('INPUTS: ', 'ID = ', id, '; ', 'AirTemp = ', AirT, '; ',  'LRI = ', LRI, '; ',  'SRI = ', avg_SRI,'; ',  'Air density = ', p)


INPUTS:  ID =  LC08_006015_20190610 ;  AirTemp =  16.07 ;  LRI =  317.2 ;  SRI =  772.98 ;  Air density =  1.219


In [3]:
# Define case area and the GEM stations

casearea_geo = ee.Geometry.Rectangle([-51.438, 64.1172, -51.2935, 64.1495])
casearea = ee.FeatureCollection(ee.Feature(casearea_geo))
casearea_frame = casearea.style(**{
    'color': '000000ff', 
    'width': 2, 
    'lineType': 'solid',
    'fillColor': '00000000'})

sub_casearea_geo = ee.Geometry.Rectangle([-51.39082, 64.127998, -51.317194, 64.147991])
sub_casearea = ee.FeatureCollection(ee.Feature(sub_casearea_geo))
sub_casearea_frame = sub_casearea.style(**{
    'color': '000000ff', 
    'width': 2, 
    'lineType': 'solid',
    'fillColor': '00000000'})



station_list = [
        ee.Feature(ee.Geometry.Point([-51.386236, 64.131]), {'name': 'InteractFen'}),
        ee.Feature(ee.Geometry.Point([-51.386008, 64.130875]), {'name': 'EddyFen'}),
        ee.Feature(ee.Geometry.Point([-51.350917, 64.135297]), {'name': 'InteractHeath'}),
        ee.Feature(ee.Geometry.Point([-51.34325, 64.133306]), {'name': 'ClimateBase'}),
        ee.Feature(ee.Geometry.Point([-51.376122, 64.133714]), {'name': 'SoilEmp'}),
        ee.Feature(ee.Geometry.Point([-51.367578, 64.133622]), {'name': 'SoilEmpSa'}),
        ee.Feature(ee.Geometry.Point([-51.385428, 64.130664]), {'name': 'SoilFen'}),
        ]
stations = ee.FeatureCollection(station_list)

# Polygon feature following the 100m contours of Copernicus DEM GLO30 - ee.ImageCollection('COPERNICUS/DEM/GLO30')
u100 = ee.Geometry.Polygon([[[
              -51.380246,
              64.149706
            ],
            [
              -51.377414,
              64.146413
            ],
            [
              -51.376985,
              64.142071
            ],
            [
              -51.369347,
              64.138926
            ],
            [
              -51.369347,
              64.136792
            ],
            [
              -51.366858,
              64.13653
            ],
            [
              -51.365055,
              64.137578
            ],
            [
              -51.347805,
              64.139488
            ],
            [
              -51.332186,
              64.140199
            ],
            [
              -51.324548,
              64.134621
            ],
            [
              -51.32163,
              64.134471
            ],
            [
              -51.310902,
              64.131138
            ],
            [
              -51.305152,
              64.131138
            ],
            [
              -51.301462,
              64.130763
            ],
            [
              -51.293823,
              64.126194
            ],
            [
              -51.293394,
              64.123235
            ],
            [
              -51.295711,
              64.122224
            ],
            [
              -51.302491,
              64.122149
            ],
            [
              -51.31588,
              64.123685
            ],
            [
              -51.320428,
              64.123685
            ],
            [
              -51.334589,
              64.126194
            ],
            [
              -51.338708,
              64.126194
            ],
            [
              -51.342828,
              64.12728
            ],
            [
              -51.359734,
              64.126082
            ],
            [
              -51.381362,
              64.126382
            ],
            [
              -51.392261,
              64.12773
            ],
            [
              -51.398268,
              64.129865
            ],
            [
              -51.40831,
              64.134995
            ],
            [
              -51.417235,
              64.138327
            ],
            [
              -51.425216,
              64.143082
            ],
            [
              -51.437832,
              64.146787
            ],
            [
              -51.437832,
              64.149744
            ],
            [
              -51.380246,
              64.149706
            ]]], None, False) 


In [4]:
# Random points for value extraction. RUNNING THIS CELL AGAIN, WILL RESULT IN NEW COLLECTION OF RANDOM POINTS AND ALTER THE RESULTS!!!
# Mute if not used!

# random_points_all = ee.FeatureCollection.randomPoints(
#     region=casearea)

# random_points_sub = ee.FeatureCollection.randomPoints(
#     region=sub_casearea)


In [5]:
# Filter the Landsat 8 collection for near-cloud-free scenes in the relevant period and area

collectionC2L2 = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filterDate('2018-01-01', '2022-01-01')
    .filter(ee.Filter.calendarRange(6,8,'month'))
    .filter(ee.Filter.lt('CLOUD_COVER_LAND', 5))
    .filterBounds(casearea)
)

collectionC2L2

In [6]:
# Open both Level 1 (TOA) and Level 2 (Surface imagery) and crop to casearea
# Level 1 will be used for the calculation of brightness temperature (band B10)
# Level 2 will be used for the extraction of atmospheric transmission (band ST_ATRAN)

Map = geemap.Map()

imageL2_all = ee.Image('LANDSAT/LC08/C02/T1_L2/(id)'.replace('(id)', id))
imageL2 = imageL2_all.clip(casearea)

imageL1_all = ee.Image('LANDSAT/LC08/C01/T1_TOA/(id)'.replace('(id)', id))
imageL1 = imageL1_all.clip(casearea)

vis_params1 = { 
    'min': 276.5, 
    'max': 298.7, 
    'palette': 'coolwarm',
    'opacity': 0.6}

vis_params2 = {
    'bands': ['ST_ATRAN'], 
    'min': 8600,
    'max': 9200,
    'palette': 'plasma', 
    'stretch': 'deviation',
    'opacity': 0.6}

Map.add_basemap('OpenTopoMap')
Map.centerObject(casearea,13)
Map.addLayer(stations, {}, 'Stations')
Map.addLayer(casearea_frame, {},'Case Area')
# Map.addLayer(imageL1, vis_params1, 'L1_B10')
Map.addLayer(imageL2, vis_params2, 'L2-ATRAN')
Map

Map(center=[64.1333647533047, -51.36575000000113], controls=(WidgetControl(options=['position', 'transparent_b…

### Calculation of landsurface temperature and conversion to outgoing langwave radiation 

Calculation of brightness temperature: Brightness temperature can either be derived from RAW imagery using the scale and correction factors, or from TOA imagery using the TOA reflectance and the conversion factor.

Here i will take a shortcut and use the TOA band 10 values, which already have been converted to brightness temperature at TOA

In [7]:
# Define parameters for LST formula 
BT = imageL1.select('B10')
AT = AirT + 273.15                                  # avg. Air temp from input in Kelvin 
#w = wv12 + wv18 / 2                                # Average water vapor content in g/cm2 at 1200h and 1800h
a10 = -55.4276                                      # Coefficient a10 for temperature range −20 °C–30 °C
b10 = 0.4086                                        # Coefficient b10 for temperature range −20 °C–30 °C
t10 = imageL2.select('ST_ATRAN').multiply(0.0001)   # Transmissivity from ST_ATRAN
E10 =  0.9843                                       # Average emissivity in 120ha test area based on ST_EMIS band
Ta = 16.0110 + 0.9262 * AT                          # Approximation of effective mean atmospheric temperature Mid-latitude summer (Wang et al., 2015)
C10 = t10.multiply(E10)                             # Wang (2015) Eq. 4
D10 = imageL2.expression(                           # Wang (2015) Eq. 5
    '(1 - t10) * (1 + (1 - E10) * t10)', {
        't10': t10,
        'E10': E10
    })

# Calculate LST (Wang et al., 2015, Eq. 3)
LST = imageL1.expression(
    '(a10 * (1 - C10 - D10) + (b10 * (1 - C10 - D10) + C10 + D10) * BT - D10 * Ta) / C10', {
    'a10': a10,
    'C10': C10,
    'D10': D10,
    'b10': b10,
    'BT': BT,
    'Ta': Ta
    }).rename('LST')

# Convert LST to outgoing longwave radiation at surface level
LRO = LST.pow(4).multiply(5.67).multiply(10**-8).multiply(E10).add((1-E10)*LRI).rename('LRO')

Map2 = geemap.Map()
Map2.add_basemap('OpenTopoMap')
Map2.centerObject(casearea,13)
Map2.addLayer(LST, {'min': 280, 'max': 300, 'palette': 'coolwarm', 'opacity':0.6}, 'LST')
# Map2.addLayer(LRO, {'min': 366, 'max': 440, 'palette': 'magma', 'opacity': 0.6}, 'Lu')
#Map2.addLayer(t10, {'min': 0.8, 'max': 1, 'palette': 'plasma'}, 't10')
#Map2.addLayer(D10, {'min': 0.1, 'max': 0.15, 'palette': 'viridis'}, 'D10')
#Map2.addLayer(C10, {'min': 0.8, 'max': 1, 'palette': 'plasma'}, 'C10')
Map2.addLayer(stations, {}, 'Stations')
Map2.addLayer(casearea_frame, {}, 'Case Area')
Map2

Map(center=[64.1333647533047, -51.36575000000113], controls=(WidgetControl(options=['position', 'transparent_b…

### Validation of land surface temperature

In [8]:
# Autom. validation using Level 2 Band 10 (LST)
LST_L2B10 = imageL2.select('ST_B10').multiply(0.00341802).add(149)
LST_validation = LST.subtract(LST_L2B10)

vis_val  = {'bands': ['LST'], 'palette': 'coolwarm', 'min': -3.0, 'max': 3.0, 'opacity': 0.6}
vis_B10LST = {'min': 260, 'max': 310, 'palette': 'coolwarm', 'opacity': 0.4}

Map3 = geemap.Map()
Map3.add_basemap('OpenTopoMap')
Map3.centerObject(casearea,13)
#Map3.addLayer(LST_L2B10, {'min': 280, 'max': 300, 'palette': 'coolwarm'}, 'LST_L2B10')
Map3.addLayer(LST_validation, vis_val, 'LST_validation')
Map3.add_colorbar(vis_val, label="LST-deviation (°C)")
Map3.addLayer(stations, {}, 'Stations')
Map3.addLayer(casearea_frame, {}, 'Casearea')
# Map3.addLayer(LST_L2B10, vis_B10LST, 'LST (K)')
# Map3.add_colorbar(vis_B10LST, label="LST (K)")

Map3

Map(center=[64.1333647533047, -51.36575000000113], controls=(WidgetControl(options=['position', 'transparent_b…

In [9]:
# Validation using GEM stations

Stations_LST = LST.reduceRegions(collection=stations, reducer=ee.Reducer.mean(), scale=30)
Stations_LRO = LRO.reduceRegions(collection=stations, reducer=ee.Reducer.mean(), scale=30)

# Create a dataframe that contains the LST and Lu values for each station
df_LST = geemap.ee_to_df(Stations_LST).rename(columns={'mean': 'LST (K)'})
df_LRO = geemap.ee_to_df(Stations_LRO).rename(columns={'mean': 'LRO (W/m2)'})
df_stations = pd.merge(df_LST, df_LRO, on='name')
df_stations = df_stations[['name', 'LST (K)', 'LRO (W/m2)']]
df_stations


,name,LST (K),LRO (W/m2)
0,InteractFen,300.558535,460.415459
1,EddyFen,299.710131,455.294859
2,InteractHeath,299.238281,452.465728
3,ClimateBase,299.817760,455.942056
4,SoilEmp,301.334888,465.139345
5,SoilEmpSa,302.856011,474.501380
6,SoilFen,299.619916,454.752913


## Calculation of shortwave radiation 

Calculate SRO using remotely sensed albedo and averaged incoming solar radiation from local weather stations (GEM). 

SRO = SRI * albedo 


In [10]:
# Calculate albedo using the atmopherically corrected data from Collection 2 Level 2. 
# Apply the appropriate scale factor and offset for each band (scale 2.75e-05 og offset -0.2, see band documentation). 

albedo = imageL2.expression(
    '(0.356 * B2 + 0.130 * B4 + 0.373 * B5 + 0.085 * B6 + 0.072 * B7 - 0.0018)', {
    'B2': imageL2.select('SR_B2').multiply(2.75e-05).add(-0.2),
    'B4': imageL2.select('SR_B4').multiply(2.75e-05).add(-0.2),
    'B5': imageL2.select('SR_B5').multiply(2.75e-05).add(-0.2),
    'B6': imageL2.select('SR_B6').multiply(2.75e-05).add(-0.2),
    'B7': imageL2.select('SR_B7').multiply(2.75e-05).add(-0.2)
    })

SRO = albedo.multiply(avg_SRI) 

vis_params_albedo = {
    'bands': ['constant'], 
    'palette': 'CMRmap', 
    'min': 0.0, 
    'max': 1.0, 
    'opacity': 0.6
    }

Map4 = geemap.Map()
Map4.centerObject(casearea, 13)
Map4.add_basemap('OpenTopoMap')
Map4.addLayer(casearea_frame, {}, 'Case Area')
Map4.addLayer(albedo, vis_params_albedo, 'Albedo')
# Map4.addLayer(SRO, {'min': 0, 'max': avg_SRI, 'palette': 'rainbow', 'opacity': 0.7}, 'SRO')
Map4.add_colorbar(vis_params_albedo, label="Albedo", position="topright")
Map4.addLayer(stations, {}, 'Stations')
Map4

Map(center=[64.1333647533047, -51.36575000000113], controls=(WidgetControl(options=['position', 'transparent_b…

### Validation of Albedo

In [11]:
# Albedo Heath GEM:     0.14
# Albedo Heath RS:      0.13 (-0.01)

# Albedo CB GEM:        0.14
# Albedo CB RS:         0.13 (-0.01)

# Albedo Fen GEM:       0.09
# Albedo Fen RS:        0.14 (+0.05)

# Relatively large difference between measured and RS Albedo for Fen. This uncertainty will affect SRO, Rnet and LE calculation. 
# 0.09 * avg_SRI = 72.5409 (SRO)
# 0.14 * avg_SRI = 112.8414 (SRO)
# avg_SRI - 72.5409 = 733.4691
# avg_SRI - 112.8414 = 693.1686    

df_albedo = geemap.ee_to_df(albedo.reduceRegions(collection=stations, reducer=ee.Reducer.mean(), scale=30)).rename(columns={'mean': 'Albedo'})
df_SRO = geemap.ee_to_df(SRO.reduceRegions(collection=stations, reducer=ee.Reducer.mean(), scale=30)).rename(columns={'mean': 'SRO (W/m2)'})

# Checking for too high albedo values that would have to be masked - none found
albedo_max = albedo.reduceRegion(reducer=ee.Reducer.max(), geometry = casearea, scale=30)





## Calculate net radiation 

Rnet = (SRI - SRO) + (LRI - LRO)

In [12]:
Rnet = imageL2.expression(
    '(SRI - SRO) + (LRI - LRO)', {
    'SRI': avg_SRI,
    'SRO': SRO,
    'LRI': LRI,
    'LRO': LRO
    }).rename('Rnet')

vis_params_Rnet = {'min': 270, 'max': 820, 'palette': 'CMRmap', 'opacity': 0.7}

Map5 = geemap.Map()
Map5.centerObject(casearea, 13)
Map5.add_basemap('OpenTopoMap')
Map5.addLayer(casearea_frame, {}, 'Case Area')
Map5.addLayer(Rnet, vis_params_Rnet, 'Rnet') # set limits back to -100 - 800
Map5.addLayer(stations, {}, 'Stations')
Map5.add_colorbar(vis_params_Rnet, label="Net radiation", position="topright")
Map5



Map(center=[64.1333647533047, -51.36575000000113], controls=(WidgetControl(options=['position', 'transparent_b…

### Validation Rnet: 


In [13]:
# Rnet Heath GEM:     543 W/m2
# Rnet Heath RS:      559 W/m2 (+16 W/m2 = 2.9%)

# Rnet CB GEM:        529 W/m2
# Rnet CB RS:         538 W/m2 (+9 W/m2 = 1.7%)

# Rnet Fen GEM:       NA
# Rnet Fen RS:        570 W/m2 (it is likely that the Rnet value is overestimated ca. +40 W/m2 due to the high albedo value.)

# It is highly problematic that LRI & LRO data from the Fen (InteractFen) is missing, leading to missing Rnet. EddyFen is the only functional 
# station that delivered validation data for flux calculations. However, the validation relies on the assumption that Rnet is correct. 

# Create a dataframe that contains Rnet values for each station
Stations_Rnet = Rnet.reduceRegions(collection=stations, reducer=ee.Reducer.mean(), scale=30)
df_Rnet = geemap.ee_to_df(Stations_Rnet).rename(columns={'mean': 'Rnet (W/m2)'})
print(df_Rnet)


   Rnet (W/m2)           name
0   536.727387    InteractFen
1   560.272746        EddyFen
2   530.637328  InteractHeath
3   521.556988    ClimateBase
4   523.576013        SoilEmp
5   507.139304      SoilEmpSa
6   553.649548        SoilFen


## Estimation of energy fluxes

### Soil heat flux

Soil heat flux is estimated using the formula 

G = 0.3 * Rns - 35 [Hoffmann et al. (2016)]

where Rns is the net radiation that reaches the soil. If necessary, the formula can be adjusted in it's slope and intercept, as suggested by Norman et al. (2000). Cristobál et al. suggest using a slope of 0.14 for arctic areas (however in permafrost), due to the insulating moss cover. 

Other than suggested by Hoffmann et al., Rns was estimated using the formula 

Rns = Rnet * (1-Pv)

where Pv is the fractional part of vegetation per cell, based on the prior NDVI calculations. The derived fraction of bare/exposed soil is then multiplied by net radiation.

 

In [14]:
# Calculate normalized difference vegetation index: (NIR - Red) / (NIR + Red).
nir_band = 'SR_B5'
red_band = 'SR_B4'
ndvi = imageL2.expression(
    '(nir - red) / (nir + red)',
    {
        'nir': imageL2.select(nir_band),
        'red': imageL2.select(red_band)
    }
).rename('nd')

vis_params_ndvi = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}

In [15]:
# Specify NDVI threshold. The threshold was manually determined by visual examination of the NDVI image. 
max_ndvi = 0.44         # Fully vegetated NDVI >0.38
min_ndvi = 0.05         # Bare soil NDVI 0.05
nd = ndvi.select('nd')  # NDVI value


Pv = ndvi.expression(
    '((ndvi - min) / (max - min))**2', {
        'ndvi': nd,
        'min': min_ndvi,
        'max': max_ndvi
    }
)

# OBS! The formula above is modified: Wang uses the global min and max values for NDVI of soil (0.2) and vegetation (0.5). In the equation 
# for the fraction of vegetation cover, Wang squarres the equation [(ndvi - min) / (max - min)]**2. This does not make sense, as it turns negative
# results to positive. The result should be a fraction of vegetation cover between 0 and 1, which is achieved by using the unsquarred formula. 

# Mask all pixels with a value > 1 - this affects around 120 pixel in the case area
Pv_masked = Pv.updateMask(Pv.lt(1)).rename('Pv')

vis_params_pv = {
  'bands': ['Pv'],
  'min': 0.0,
  'max': 1.0,
  'palette': 'Greens', 
  'opacity': 1.0}

Map6 = geemap.Map()
Map6.centerObject(casearea, 13)
Map6.add_basemap('OpenTopoMap')
Map6.add_layer(ndvi, vis_params_ndvi, 'NDVI')
Map6.addLayer(Pv_masked, vis_params_pv, 'Vegetation cover')
Map6.add_colorbar(vis_params_pv, label="Fractional vegetation cover")
Map6.addLayer(casearea_frame, {}, 'Case Area')
Map6.addLayer(stations, {}, 'Stations')
Map6


Map(center=[64.1333647533047, -51.36575000000113], controls=(WidgetControl(options=['position', 'transparent_b…

In [16]:
# Calculate soil heat flux for all pixels. 
# OBS: The PV will likely change due to phenological changes in vegetation cover. Though this should not affect the calculation, as the 
# vegetation will loose its leaves, allowing more radiation to pass through the canopy and reach the soil.

G1 = Rnet.expression(
    '0.30 * (Rnet * (1 - Pv)) - 35', {
        'Rnet': Rnet,
        'Pv': Pv_masked
    }
)

G2 = Rnet.expression(
    '0.14 * (Rnet * (1 - Pv)) - 35', {
        'Rnet': Rnet,
        'Pv': Pv_masked
    }
).rename('G')

vis_params_G = {
    'min': -35,
    'max': 80,
    'palette': 'gist_earth',
    'opacity': 1
}

Map7 = geemap.Map()
Map7.centerObject(casearea, 13)
Map7.add_basemap('OpenTopoMap')
Map7.addLayer(casearea_frame, {}, 'Case Area')
Map7.addLayer(G2, vis_params_G, 'G')
Map7.add_colorbar(vis_params_G, label="Soil heat flux (W/m2)")
Map7.addLayer(stations, {}, 'Stations')
Map7


Map(center=[64.1333647533047, -51.36575000000113], controls=(WidgetControl(options=['position', 'transparent_b…

In [17]:
# Validation using GEM stations
# DATE: 2021-06-22

Stations_G1 = G1.reduceRegions(collection=stations, reducer=ee.Reducer.mean(), scale=30)
Stations_G2 = G2.reduceRegions(collection=stations, reducer=ee.Reducer.mean(), scale=30)

# Create a dataframe that contains the G1 and G2 values for each station
df_G1 = geemap.ee_to_df(Stations_G1)
df_G2 = geemap.ee_to_df(Stations_G2)
df_G = pd.merge(df_G1, df_G2, on='name')
df_G = df_G.rename(columns={'mean_x': 'G1 (W/m2)', 'mean_y': 'G2 (W/m2)'})
df_G


,G1 (W/m2),name,G2 (W/m2)
0,110.904261,InteractFen,33.088655
1,118.746375,EddyFen,36.748308
2,71.886542,InteractHeath,14.880386
3,107.513445,ClimateBase,31.506274
4,86.490158,SoilEmp,21.695407
5,59.437961,SoilEmpSa,9.071049
6,113.093330,SoilFen,34.110221


### Sensible Heat Flux

Sensible heat flux is estimated using the forumla from Hoffmann et al. (2016) 

H = p * cp * ((Ts-Ta/Rs))

where Cp is the specific heat of the air Cp = ca. 1005 J/kg/K. P (ro) is the air density p = Pressure (Pa) / (Gas-Constant (287 J/K/Kg) * Temperature (K)). Ts is the remotly sensed surface temperature. Ta is the averaged ambient temperature of three in-situ stations (InteractFen, Interactheath, ClimateBase station). Rs is a specific resistance value. These values are normally canclulated by xxx (see Hoffmann et al., 2016 and Norman et al., 2000). For the ease of computation, the Rs was set to a contant value of XXX, as suggested as a well fitting standard value in literature (T.R. Oke, Boundary Layer Climates).

On 22.06.2021:
p = 100350.00 / (287 * 288.65) = 1211 kg/m3 

In [18]:
cp = 1005                   # specific heat of the air, relatively constant at 1005 J/kg/K.
Rs = 46

H = LST.expression(
    'p*cp*((LST - AirT)/Rs)', {
        'p': p,
        'cp': cp,
        'LST': LST,
        'AirT': AirT+273.15,
        'Rs': Rs
    }
).rename('H')

vis_params_H = {
    'min': -350,
    'max': 450,
    'palette': 'gist_earth',
    'opacity': 1}

Map8 = geemap.Map()
Map8.centerObject(casearea, 13)
Map8.add_basemap('OpenTopoMap')
Map8.addLayer(casearea_frame, {}, 'Case Area')
Map8.addLayer(H, vis_params_H, 'H')
Map8.add_colorbar(vis_params_H, label="Sensible Heat Flux (W/m2)")
Map8.addLayer(stations, {}, 'Stations')
Map8


Map(center=[64.1333647533047, -51.36575000000113], controls=(WidgetControl(options=['position', 'transparent_b…

In [19]:
# Validation using GEM stations

#Extract the sensible heat flux only for the EddyFen station
Stations_H = H.reduceRegions(collection=stations, reducer=ee.Reducer.mean(), scale=30)
df_H = geemap.ee_to_df(Stations_H)
df_H = df_H.rename(columns={'mean': 'H (W/m2)'})
df_H

# A sensitivity analysis was performed. The resistance value was adjusted from 46 s/m, to match the measured sensible heat flux from EddyHeath. 
# Unfortunately there is no other data to validate the adjustment. 

,H (W/m2),name
0,301.973525,InteractFen
1,279.378418,EddyFen
2,266.811873,InteractHeath
3,282.244837,ClimateBase
4,322.649748,SoilEmp
5,363.161066,SoilEmpSa
6,276.975776,SoilFen


### Latent Energy 

Using equation Rnet = G + H + LE, latent energy can be isolated as the last term in the equation:

LE = Rnet - G - H

In [20]:
LE = Rnet.subtract(H).subtract(G2).rename('LE')

vis_params_LE = {
    'palette': 'gist_earth', 
    'min': 0,
    'max': 1200,
    'opacity': 1
    }

Map9 = geemap.Map()
Map9.centerObject(casearea, 13)
Map9.add_basemap('OpenTopoMap')
Map9.addLayer(casearea_frame, {}, 'Case Area')
Map9.addLayer(LE, vis_params_LE, 'LE')
Map9.add_colorbar(vis_params_LE, label="Latent Heat Flux (W/m2)")
Map9.addLayer(stations, {}, 'Stations')
Map9

Map(center=[64.1333647533047, -51.36575000000113], controls=(WidgetControl(options=['position', 'transparent_b…

In [21]:
# Validation using GEM stations

#Extract the latent heat flux for the GEM stations
Stations_LE = LE.reduceRegions(collection=stations, reducer=ee.Reducer.mean(), scale=30)
df_LE = geemap.ee_to_df(Stations_LE)
df_LE = df_LE.rename(columns={'mean': 'LE (W/m2)'})
df_LE

,LE (W/m2),name
0,201.665207,InteractFen
1,244.146019,EddyFen
2,248.945069,InteractHeath
3,207.805876,ClimateBase
4,179.230858,SoilEmp
5,134.907189,SoilEmpSa
6,242.563552,SoilFen


# Validation

In [22]:
df_validation = df_stations.merge(df_Rnet, on='name').merge(df_G, on='name').merge(df_H, on='name').merge(df_LE, on='name').merge(df_albedo, on='name').merge(df_SRO, on='name')
# export the validation data to a csv file
df_validation.to_csv('(date)_results_RS.csv'.replace('(date)', date).replace('/', ''))
df_validation


,name,LST (K),LRO (W/m2),Rnet (W/m2),G1 (W/m2),G2 (W/m2),H (W/m2),LE (W/m2),Albedo,SRO (W/m2)
0,InteractFen,300.558535,460.415459,536.727387,110.904261,33.088655,301.973525,201.665207,0.120362,93.037154
1,EddyFen,299.710131,455.294859,560.272746,118.746375,36.748308,279.378418,244.146019,0.096526,74.612395
2,InteractHeath,299.238281,452.465728,530.637328,71.886542,14.880386,266.811873,248.945069,0.138525,107.076944
3,ClimateBase,299.817760,455.942056,521.556988,107.513445,31.506274,282.244837,207.805876,0.145775,112.680957
4,SoilEmp,301.334888,465.139345,523.576013,86.490158,21.695407,322.649748,179.230858,0.131264,101.464642
5,SoilEmpSa,302.856011,474.501380,507.139304,59.437961,9.071049,363.161066,134.907189,0.140417,108.539316
6,SoilFen,299.619916,454.752913,553.649548,113.093330,34.110221,276.975776,242.563552,0.105795,81.777539


In [23]:
# Load raster land cover classification by Rudd, reproject to other imagery and cut to case area (adjust extent)

LC = ee.Image('projects/ee-ivanburgov666/assets/Kobbefjord_Landcover_UTM84_22N_DAR').reproject(crs=Rnet.projection(), scale=10).clip(casearea)
LC_sub = LC.clip(u100)

vis_params_LC = {
    'bands': ['b1'],
    'palette': ['#C2C2C2', '#F7F497', 'FF8F8F', '#A5FF75', '#1E5700', '#6C007A', '#4D4D4D', '#000594', '#FFFFFF'],
    'min': 1.0, 
    'max': 9.0, 
    'opacity': 0.68, 
}

lc_labels = ['Barren ground', 'Abrasion surfaces', 'Fen', 'Dry Heath and Grassland', 'Wet Heath', 'Copse and Tall Shrubs', 'Shadow', 'Water', 'Snow']
lc_colours = [('#C2C2C2'), ('#F7F497'), ('#FF8F8F'), ('#A5FF75'), ('#1E5700'), ('#6C007A'), ('#4D4D4D'), ('#000594'), ('#FFFFFF')]

# Convert raster LC to vector layer in order to use the reducer tool. Scale remains at 10m for a finer depiction of LC polygones. 
vectors = LC.reduceToVectors(crs=Rnet.projection(), scale=10, geometryType='polygon', eightConnected=False, labelProperty='class')
vectors_sub = LC_sub.reduceToVectors(crs=Rnet.projection(), scale=10, geometryType='polygon', eightConnected=False, labelProperty='class')

Map10 = geemap.Map()
Map10.centerObject(casearea, 13)
Map10.add_basemap('OpenTopoMap')
Map10.addLayer(LC, vis_params_LC, 'Land Cover')
Map10.addLayer(vectors_sub, {}, 'vector zones') #changed from vectors to sub
Map10.addLayer(casearea_frame, {}, 'Case Area')
# Map10.addLayer(sub_casearea_frame, {}, 'Sub Case Area')
# Map10.addLayer(u100, {}, 'u100')
Map10.addLayer(stations, {}, 'Stations')
# Map10.addLayer(imageL2, vis_params2, 'L2-ATRAN')
Map10.add_legend(title = "Land cover classification", labels = lc_labels, colors = lc_colours)

Map10

Map(center=[64.1333647533047, -51.36575000000113], controls=(WidgetControl(options=['position', 'transparent_b…

In [24]:
#Calculate the size of the casearea
size_whole = casearea.geometry().area().divide(10000).getInfo()
size_sub = sub_casearea.geometry().area().divide(10000).getInfo()

print(size_whole)
print(size_sub)

2517.7023390014624
793.9091681664138


In [25]:
print(LC.projection().getInfo())
print(Rnet.projection().getInfo())
print(LST.projection().getInfo())
print(H.projection().getInfo())
print(casearea_geo.projection().getInfo())
print(sub_casearea_geo.projection().getInfo())
print(u100.projection().getInfo())
print(imageL1_all.select('B10').projection().getInfo())
print(imageL2_all.select('ST_ATRAN').projection().getInfo())

# I set eight-connected to 'False' - produces more polygones, as diagonal neighbours don't count. T
# Therefore more accuracy when using the reducer tool (when averaging environmental variables per polygon). 
# Yields 2333 polygones when 'True', Vs. 3647 when 'False'.


{'type': 'Projection', 'crs': 'EPSG:32622', 'transform': [10, 0, 398985, 0, 10, 7246815]}
{'type': 'Projection', 'crs': 'EPSG:32622', 'transform': [30, 0, 398985, 0, -30, 7246815]}
{'type': 'Projection', 'crs': 'EPSG:32622', 'transform': [30, 0, 398985, 0, -30, 7246815]}
{'type': 'Projection', 'crs': 'EPSG:32622', 'transform': [30, 0, 398985, 0, -30, 7246815]}
{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}
{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}
{'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}
{'type': 'Projection', 'crs': 'EPSG:32622', 'transform': [30, 0, 398985, 0, -30, 7246815]}
{'type': 'Projection', 'crs': 'EPSG:32622', 'transform': [30, 0, 398985, 0, -30, 7246815]}


In [26]:
# Map for visual comparison of LC and distribution of LE, H, G 

Map11 = geemap.Map()
Map11.centerObject(casearea, 13)
Map11.add_basemap('OpenTopoMap')
Map11.addLayer(LC, vis_params_LC, 'Land Cover')
Map11.addLayer(casearea_frame, {}, 'Case Area')
Map11.addLayer(Rnet, vis_params_Rnet, 'Rnet')
Map11.addLayer(H, vis_params_H, 'H')
Map11.addLayer(LE, vis_params_LE, 'LE')
Map11.addLayer(G2, vis_params_G, 'G')
# Map11.addLayer(LST_validation, vis_val, 'LST_validation')
Map11.add_legend(title = "Land cover classification", labels = lc_labels, colors = lc_colours)
Map11

# LC ~ H :  No visual correlation
# LC ~ LE : No visual correlation
# LC ~ G :  Visual correlation, especially for Copse and Tall Shrubs, Abrasion surfaces, likely also dry heath and grassland and wet heath. 

Map(center=[64.1333647533047, -51.36575000000113], controls=(WidgetControl(options=['position', 'transparent_b…

In [27]:
# Count number of polygones per class 
df_vec = geemap.ee_to_df(vectors)
print(df_vec.groupby('class').size())
# print(df_vec.value_counts('label')) # This one puts them in descending order

# Calculate the area of each land class within case area
df_vec_area = df_vec.groupby('class').sum()
print(df_vec_area)

# Calculate the area of each land class within case area in square meters (vectors are based on raster data with 10x10 m resolution)
df_vec_area_m2 = df_vec_area * 10 * 10
print(df_vec_area_m2)

# Calculate the fraction of each land class within case area
df_vec_percentage1 = (df_vec_area_m2 / casearea_geo.area().getInfo()) *100
print(df_vec_percentage1)

df_vec_percentage1 = (df_vec_area / df_vec_area.sum()) *100
print(df_vec_percentage1)



class
1     953
2    2644
3     476
4    2088
5    1063
6     236
7     101
8      23
9      29
dtype: int64
       count
class       
1      75982
2      28634
3       4262
4      47472
5      60970
6       4519
7       2246
8      29238
9        852
         count
class         
1      7598200
2      2863400
3       426200
4      4747200
5      6097000
6       451900
7       224600
8      2923800
9        85200
           count
class           
1      30.179104
2      11.373068
3       1.692813
4      18.855287
5      24.216524
6       1.794890
7       0.892083
8      11.612969
9       0.338404
           count
class           
1      29.893577
2      11.265467
3       1.676797
4      18.676896
5      23.987410
6       1.777909
7       0.883643
8      11.503098
9       0.335202


# Value extraction and data export

Run following cell only if data extraction is needed. Unmute the desired sampling stategy.

In [28]:
# Create flux data raster stack for entire case area and subarea
# fluxes = G2.addBands(H).addBands(LE).addBands(Rnet)
fluxes = G2.addBands(H).addBands(LE).addBands(Rnet).addBands(LC).addBands(albedo)
fluxes_subarea = fluxes.clip(sub_casearea)

#####  CHOOSE SAMPLING STRATEGY #####

### POLYGONES ###
### Dataset with mean values per polygone (7612 in whole casearea and 2993 in subarea)
fluxes_reduced_all = fluxes.reduceRegions(collection=vectors, reducer=ee.Reducer.mean(), scale=10)
fluxes_reduced_subarea = fluxes_subarea.reduceRegions(collection=vectors_sub, reducer=ee.Reducer.mean(), scale=10)

df_fluxes_all = geemap.ee_to_df(fluxes_reduced_all)
print(df_fluxes_all)
df_fluxes_subarea = geemap.ee_to_df(fluxes_reduced_subarea)
print(df_fluxes_subarea)

df_fluxes_all.to_csv('(date)_fluxes_all.csv'.replace('(date)', date).replace('/',''), index=False)
df_fluxes_subarea.to_csv('(date)_fluxes_subarea.csv'.replace('(date)', date).replace('/',''), index=False)


### PIXELS ###
### Data with the actual flux value per pixel (should be muted - takes 4 minutes and generates a 18MB csv) 
# pixel_all = fluxes.sample(**{
#   'region': casearea,
#   'scale': 10,
#   'geometries': True 
# })

# pixel_sub = fluxes_subarea.sample(**{
#   'region': sub_casearea,
#   'scale': 10, 
#   'geometries': True 
# })

# df_pixels_all = geemap.ee_to_df(pixel_all)
# print(df_pixels_all)
# df_pixels_sub = geemap.ee_to_df(pixel_sub)
# print(df_pixels_sub)

# df_pixels_all.to_csv('(date)_pixels_all.csv'.replace('(date)', date).replace('/',''), index=False) #mute
# df_pixels_sub.to_csv('(date)_pixels_sub.csv'.replace('(date)', date).replace('/',''), index=False) #mute


### RANDOM POINTS ###
### Dataset with cell values of 1000 random points
# rp_reduced_all = fluxes.reduceRegions(collection=random_points_all, reducer=ee.Reducer.mean(), scale=10)
# rp_reduced_sub = fluxes_subarea.reduceRegions(collection=random_points_sub, reducer=ee.Reducer.mean(), scale=10)

# df_rp_all = geemap.ee_to_df(rp_reduced_all)
# df_rp_sub = geemap.ee_to_df(rp_reduced_sub)

# df_rp_all.to_csv('(date)_rp_all.csv'.replace('(date)', date).replace('/',''), index=False)
# df_rp_sub.to_csv('(date)_rp_sub.csv'.replace('(date)', date).replace('/',''), index=False)